# Configurações

## Importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

## Definição do Dataframe

In [2]:
df = pd.read_csv('../datasets/data_clean.csv')

# Pré-processamento

## Seleção de Features

In [3]:
df_selecao = df.copy()

In [4]:
df_selecao.drop(columns=['artist', 'song', 'duration_ms', 'year', 'key', 'mode', 'tempo', 'explicit', 'loudness', 'valence', 'genre', 'liveness'], inplace=True)

## Tratamento da feature "genre"

A feature "genre" possui os gêneros das músicas, sendo uma string, com todos os gêneros da música, separados por vírgula. 

O objetivo desse pré-processamento é dividir essa string e aplicar uma técnica para criar colunas binárias dos gêneros.

In [5]:
df_pp = df.copy()

In [6]:
mlb = MultiLabelBinarizer()

In [7]:
df_pp = df[~df['genre'].astype(str).str.contains('set()', case=False, na=False)].copy()

/tmp/ipykernel_6386/3102771490.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_pp = df[~df['genre'].astype(str).str.contains('set()', case=False, na=False)].copy()


In [8]:
df_pp['genre'] = df_pp['genre'].apply(lambda x: x.replace(', ', ',').split(','))

In [9]:
encoded = np.array(mlb.fit_transform(df_pp['genre']))

In [10]:
encoded_df = pd.DataFrame(encoded, columns=mlb.classes_)

In [11]:
encoded_df.columns

Index(['Dance/Electronic', 'Folk/Acoustic', 'R&B', 'World/Traditional',
       'blues', 'classical', 'country', 'easy listening', 'hip hop', 'jazz',
       'latin', 'metal', 'pop', 'rock'],
      dtype='object')

In [12]:
encoded_df = pd.DataFrame(encoded, columns=['genre_' + c.lower().replace(' ', '_') for c in mlb.classes_])

In [13]:
df_pp_processado = pd.concat([df_pp, encoded_df], axis=1).drop(columns=['genre'])

In [14]:
df_pp_processado

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,...,genre_blues,genre_classical,genre_country,genre_easy_listening,genre_hip_hop,genre_jazz,genre_latin,genre_metal,genre_pop,genre_rock
0,Britney Spears,Oops!...I Did It Again,211160.0,False,2000.0,77.0,0.751,0.834,1.0,-5.444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,blink-182,All The Small Things,167066.0,False,1999.0,79.0,0.434,0.897,0.0,-4.918,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Faith Hill,Breathe,250546.0,False,1999.0,66.0,0.529,0.496,7.0,-9.007,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Bon Jovi,It's My Life,224493.0,False,2000.0,78.0,0.551,0.913,0.0,-4.063,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,*NSYNC,Bye Bye Bye,200560.0,False,2000.0,65.0,0.614,0.928,8.0,-4.806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


# Gerar o dataframe

In [15]:
df.to_csv('../datasets/pre_processing.csv', index=False, encoding='utf-8')

# Resultados

Foram removidas 12 features e mantido apenas 6.